# Coloc
This notebook take a list of rds output from susie_rss and a list of rds output from uni_susie to conduct coloc on them.

In [ ]:
sos run pipeline/coloc.ipynb coloc \
    --qtl_rds `ls susie/*rds` \
    --sumstat_rds `ls output/*rds` \
    --region_list test.region_list
    --container containers/stephenslab.sif &

In [ ]:
[global]
parameter: container = ""
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20
parameter: cwd = path("output")
# Get the susie_rss results
parameter: sumstat_rds = paths
# Get the susie results
parameter: qtl_rds = paths
# A region_list to link the susie results and the rss results
parameter: region_list
import pandas as pd

sumstat_list = pd.DataFrame({"ss_path" : [str(x) for x in sumstat_rds]})
sumstat_list["#chr"] = [x.split(".")[-4].split("_")[0].replace("chr","")  for x in sumstat_list.path ]
sumstat_list["from"] = [x.split(".")[-4].split("_")[1]  for x in sumstat_list.path ]
sumstat_list["to"] = [x.split(".")[-4].split("_")[2]  for x in sumstat_list.path ]

qtl_list = pd.DataFrame({"qtl_path" : [str(x) for x in qtl_rds], "gene_id" = [str(x).split(".")[-4] for x in qtl_rds] })

region_list = region_list.merge(qtl_list, on = "gene_id").merge(sumstat_list , on = "#chr")
region_list = region_list.query("start > from & end < to")
ss_input = region_list.ss_path
qlt_input = region_list.qtl_path



In [ ]:
[coloc]
input: ss_input, group_with = "qlt_input", group_by = 1
output: f'{cwd}/{_input[0]:nnn}.{_input[1]:nnn}.coloc.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr", container = container
    gwas <- readRDS(file = "${_input[0]}")
    eqtl <- readRDS(file = "${_input[1]}")
    coloc_result = list()
    for(i in 1:length(eqtl) ){coloc_result[i] <- coloc.susie(gwas, eqtl[i]) }
    coloc_result%>%saveRDS("${_output}")
    